<a href="https://colab.research.google.com/github/Xornotor/Trabalho_IA_2022-2/blob/main/AndrePaiva_CarlosCerqueira_Trabalho_IA_2022_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Trabalho Final: MATA64 - Inteligência Artificial - UFBA 2022.2**
# **Alunos:** André Paiva e Carlos Cerqueira

## 1 - Instalação e configuração de dependências

Utilizamos a biblioteca **Manim** para facilitar a visualização dos algoritmos desenvolvidos. O seguinte trecho de código deve ser executado e após sua execução o Runtime deve ser reinicializado.

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg texlive texlive-latex-extra texlive-fonts-extra texlive-latex-recommended texlive-science tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

Abaixo, são importadas as bibliotecas utilizadas neste Notebook.

In [ ]:
import numpy as np
import pandas as pd
from manim import *
from random import random

Manim Community v0.16.0.post0

## 2 - Busca A* Aplicada a determinação de rota de rede

## 3 - Perceptron aplicado à predição de preços de casas

Criamos um modelo de **perceptron** utilizando um dicionário, cujas chaves são *W* e *b*, sendo que:


*   `perceptron['W']` retorna um `np.array` contendo os pesos;
*   `perceptron['b']` retorna o bias.


Foram definidas as funções `create_perceptron`, `predict` e `train_step`, cujas respectivas funções são: 


*   Criar um perceptron e inicializar seus pesos e bias;
*   Efetuar a predição do valor esperado com base em features de entrada;
*   Efetuar o treinamento dos pesos e bias do perceptron.




In [ ]:
# Função de inicialização do perceptron com pesos e bias aleatórios
def create_perceptron(features):
  perceptron = {}
  perceptron['W'] = np.array([random() for i in range(features)])
  perceptron['b'] = random()
  return perceptron

# Função de predição do perceptron
def predict(input, perceptron):
  return np.dot(perceptron['W'], input) + perceptron['b']

# Função de treinamento do perceptron
def train_step(input, output, perceptron, learning_rate=1e-5):
  pred = predict(input, perceptron)
  perceptron['W'] = perceptron['W'] + (learning_rate * (output - pred) * input)
  perceptron['b'] = perceptron['b'] + (learning_rate * (output - pred))
  return perceptron

A seguir, temos dois dataframes com dados de algumas casas à venda, sendo um dataframe de **treino** e um dataframe de **testes**.

O data frame de **treino** contém as seguintes informações:
*   Área da casa (m²);
*   Idade da construção (anos);
*   Número de Quartos;
*   Número de Banheiros;
*   Preço.

O data frame de **testes** contém:
*   Área da casa (m²);
*   Idade da construção (anos);
*   Número de Quartos;
*   Número de Banheiros.



In [ ]:
#Dataframe de treinamento
train_df = pd.DataFrame(np.array([
                            [287.00, 20, 3, 3, 500000.00],
                            [57.00, 30, 2, 1, 100000.00],
                            [89.00, 22, 3, 2, 225000.00],
                            [157.00, 40, 4, 3, 260000.00],
                            [210.00, 60, 5, 3, 275000.00],
                            [250.00, 28, 4, 2, 330000.00]
                          ]),                        
                        columns=['Área','Idade','Quartos','Banheiros','Preço']
                        )

#Dataframe de teste
test_df = pd.DataFrame(np.array([
                            [242.00, 23, 4, 2],
                            [67.00, 32, 2, 1],
                            [94.00, 27, 3, 3]
                          ]),                        
                        columns=['Área','Idade','Quartos','Banheiros']
                        )

In [ ]:
#Exibição do dataframe de treino
train_df

,Área,Idade,Quartos,Banheiros,Preço
0,287.0,20.0,3.0,3.0,500000.0
1,57.0,30.0,2.0,1.0,100000.0
2,89.0,22.0,3.0,2.0,225000.0
3,157.0,40.0,4.0,3.0,260000.0
4,210.0,60.0,5.0,3.0,275000.0
5,250.0,28.0,4.0,2.0,330000.0


In [ ]:
#Exibição do dataframe de testes
test_df

,Área,Idade,Quartos,Banheiros
0,242.0,23.0,4.0,2.0
1,67.0,32.0,2.0,1.0
2,94.0,27.0,3.0,3.0


A seguir, é feita a criação e inicialização de um perceptron e a rotina de treinamento.

In [ ]:
#Hiperparâmetros
ITERATIONS = 5

#Valores úteis
examples = train_df.shape[0]
features = train_df.shape[1] - 1

#Data stream (para visualização)
data_stream = {'input': [],
               'perceptron': [],
               'output': []}

#Inicialização do perceptron
perceptron = create_perceptron(features)

#Treinamento
for i in range(ITERATIONS):
  for j in range(examples):
    input = train_df.iloc[j, 0:features].to_numpy()
    output = train_df.iloc[j, train_df.shape[1] - 1]
    data_stream['input'].append(input.tolist())
    data_stream['output'].append(output)
    data_stream['perceptron'].append(perceptron.copy())
    perceptron = train_step(input, output, perceptron)
    if(i == ITERATIONS - 1 and j == examples - 1):
      data_stream['input'].append(input.tolist())
      data_stream['output'].append(output)
      data_stream['perceptron'].append(perceptron.copy())

Após o treinamento, mostramos novamente o dataset de treino, agora com adição da coluna de preços preditos pelo perceptron. Podemos notar uma discrepância significativa, apesar de uma relativa proximidade entre os valores real e predito.

In [ ]:
#Predições do dataframe de treinamento
train_predictions = np.zeros((train_df.shape[0]))

for j in range(train_df.shape[0]):
    input = train_df.iloc[j, 0:features].to_numpy()
    train_predictions[j] = predict(input, perceptron)

train_df_with_predictions = train_df
train_df_with_predictions['Preço predito'] = train_predictions

train_df_with_predictions

,Área,Idade,Quartos,Banheiros,Preço,Preço predito
0,287.0,20.0,3.0,3.0,500000.0,402500.843505
1,57.0,30.0,2.0,1.0,100000.0,78311.781801
2,89.0,22.0,3.0,2.0,225000.0,123853.831950
3,157.0,40.0,4.0,3.0,260000.0,218384.384499
4,210.0,60.0,5.0,3.0,275000.0,291672.695397
5,250.0,28.0,4.0,2.0,330000.0,349940.809256


Agora, mostramos novamente o dataset de testes, mas agora com as predições feitas pelo perceptron após o treinamento.

In [ ]:
#Predições do dataframe de testes
test_predictions = np.zeros((test_df.shape[0]))

for j in range(test_df.shape[0]):
    input = test_df.iloc[j].to_numpy()
    test_predictions[j] = predict(input, perceptron)

test_df_with_predictions = test_df
test_df_with_predictions['Preço predito'] = test_predictions

test_df_with_predictions

,Área,Idade,Quartos,Banheiros,Preço predito
0,242.0,23.0,4.0,2.0,339005.161241
1,67.0,32.0,2.0,1.0,92250.808413
2,94.0,27.0,3.0,3.0,130586.411849


Agora, veremos uma animação ilustrando o processo de treinamento do perceptron e atualização dos pesos e bias.

In [ ]:
%%manim -v WARNING --disable_caching -qm ManimPerceptron

PERCEP_RADIUS = 1.3
RENDER_STEPS = 3

class InputGraph:
    def __init__(self, arrow_num, input_num=0, weight_num=0):
      self.arrow = Arrow(color=BLUE, start=(-6, 3 - (6*arrow_num/(features - 1)), 0), end=ORIGIN, buff=PERCEP_RADIUS)
      self.weight_tracker = ValueTracker(weight_num)
      self.weight_var = DecimalNumber(0, color=BLUE, font_size=28, include_sign=True, group_with_commas=False).next_to(self.arrow.get_midpoint(), UP).add_updater(lambda wei: wei.set_value(self.weight_tracker.get_value()))
      self.input_tracker = ValueTracker(int(input_num))
      self.input_var = DecimalNumber(0, font_size=28, group_with_commas=False).next_to(self.arrow.start).align_to(self.arrow.start, LEFT).add_updater(lambda inp: inp.set_value(self.input_tracker.get_value()))

    def get_graphics(self):
      return (self.arrow, self.input_var, self.weight_var)

    def get_input_tracker(self):
      return self.input_tracker

    def get_weight_tracker(self):
      return self.weight_tracker

    def set_input_tracker(self, inp):
      self.input_tracker.set_value(inp)

    def set_weight_tracker(self, wei):
      self.weight_tracker.set_value(wei)

class ManimPerceptron(Scene):
  def construct(self):
    input_values = data_stream['input'][0]
    weight_values = data_stream['perceptron'][0]['W'].tolist()
    bias_value = data_stream['perceptron'][0]['b']

    circle = Circle(color=WHITE, radius=PERCEP_RADIUS)
    percep_formula = MathTex(r"\sum_{i} w_i \cdot x_{i} + b", font_size=30)

    bias_arrow = Arrow(color=GREEN, start=(0,4,1), end=ORIGIN, buff=PERCEP_RADIUS)
    bias_tracker = ValueTracker(bias_value)
    bias_var = DecimalNumber(0, font_size=28, color=GREEN, include_sign=True, group_with_commas=False)
    bias_var.add_updater(lambda x: x.set_value(float(bias_tracker.get_value())))
    bias_var.next_to(bias_arrow)

    output_arrow = Arrow(color=ORANGE, start=ORIGIN, end=(5, 0, 0), buff=PERCEP_RADIUS)
    output_tracker = ValueTracker(0)
    output_var = DecimalNumber(0, num_decimal_places=2, font_size=32, include_sign=False, group_with_commas=False)
    output_var.add_updater(lambda x: x.set_value(float(output_tracker.get_value())))
    output_var.next_to(output_arrow.end, buff=0)

    prediction = DecimalNumber(0, num_decimal_places=2, font_size=36, include_sign=False, group_with_commas=False).arrange(RIGHT, center=True)
    prediction_tracker = ValueTracker(0)
    prediction.add_updater(lambda x: x.set_value(float(prediction_tracker.get_value())))
    
    expected = DecimalNumber(0, num_decimal_places=2, font_size=32, include_sign=False, group_with_commas=False, color=RED).next_to(output_var, DOWN)
    expected_tracker = ValueTracker(0)
    expected.add_updater(lambda x: x.set_value(float(expected_tracker.get_value())))

    backprop_arrow = ArcBetweenPoints((4.5, -1.7, 0), (-3, -3, 0), radius=-10, color=RED)
    backprop_arrow.add_tip()
    backprop_w = MathTex(r"w_i = w_i + \eta \cdot(y_{real} - y_{predito}) \cdot x_i", font_size=26, color=RED)
    backprop_b = MathTex(r"b_i = b_i + \eta \cdot(y_{real} - y_{predito})", font_size=26, color=RED)
    backprop_b.next_to(backprop_arrow.get_midpoint(), UP, buff=0.7)
    backprop_w.next_to(backprop_b, UP, 0.2)

    self.play(Create(circle), Write(percep_formula))

    input_graph_list = []
    for i in range(features):
      input_graph_list.append(InputGraph(i, input_values[i], weight_values[i]))
      arrow, input_var, weight_var = input_graph_list[i].get_graphics()
      self.play(GrowArrow(arrow), Write(input_var), Write(weight_var), run_time=0.12)
             
    self.play(Write(bias_var), Write(output_var), GrowArrow(bias_arrow), GrowArrow(output_arrow), run_time=0.12)

    self.wait(5)

    self.play(FadeTransform(percep_formula, prediction), run_time=0.5)

    self.wait(5)

    for i in range(1, min(RENDER_STEPS + 1, len(data_stream['perceptron']))):
      for j in range(features):
        arrow, input_var, weight_var = input_graph_list[j].get_graphics()
        self.play(FadeTransform(input_var, prediction),
                  FadeTransform(weight_var, prediction),
                  prediction_tracker.animate.increment_value(input_var.get_value()*weight_var.get_value()),
                  run_time=0.75)
        input_graph_list[j].set_input_tracker(data_stream['input'][i][j])
        input_graph_list[j].set_weight_tracker(data_stream['perceptron'][i]['W'][j])
        self.wait(1.5)
      self.play(FadeTransform(bias_var, prediction),
                prediction_tracker.animate.increment_value(bias_var.get_value()),
                run_time=0.75)
      self.wait(1.5)
      bias_tracker.set_value(data_stream['perceptron'][i]['b'])
      self.play(FadeTransform(prediction, output_var),
                output_tracker.animate.set_value(prediction_tracker.get_value()))
      expected_tracker.set_value(data_stream['output'][i])
      self.wait(1)
      self.play(Write(expected), run_time=0.12)
      self.play(Indicate(expected), run_time=0.5)
      self.play(Write(backprop_arrow), Write(backprop_b), Write(backprop_w))
      self.wait(1)
      prediction_tracker.set_value(0)
      for j in range(features):
        arrow, input_var, weight_var = input_graph_list[j].get_graphics()
        self.play(Write(input_var), Write(weight_var), run_time=0.12)
      self.play(Write(bias_var), run_time=0.12)
      self.wait(4)
      self.play(FadeOut(expected), FadeOut(backprop_arrow), FadeOut(backprop_b), FadeOut(backprop_w), Write(prediction), run_time=0.5)
      self.wait(2)
    
    self.wait(3)